# <span style="color: orange;">Analyse Exploratoire des Données (EDA) - Prédiction du Taux de Grippe</span>

---
### <span style="color: green;">Introduction</span> 

**Objectif du projet** : Prédire le taux de grippe pour 100 000 habitants par région française pour des semaines spécifiques.

**Description du dataset** : Le fichier `train.csv` est le résultat d'une fusion entre :
- Les données d'entraînement originales contenant les informations sur les cas de grippe par région et par semaine
- Des données démographiques et de requêtes Google par région




---

### <span style="color: green;">Import des Librairies</span>

In [103]:
# Librairies de base
import pandas as pd
import numpy as np
# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
# Style des graphiques
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
# Ignorer les warnings
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append("../src")
from eda import *

### <span style="color: green;">Chargement du Dataset</span>

In [104]:
df = pd.read_csv('../data/processed/train.csv')
print(f"Dimensions : {df.shape[0]} lignes × {df.shape[1]} colonnes")
df.tail()

Dimensions : 9196 lignes × 13 colonnes


,Id,week,region_name,TauxGrippe,pop_0_19,pop_20_39,pop_40_59,pop_60_74,pop_75_plus,pop_total,requete_grippe,requete_grippe_aviaire_vaccin,requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie
9191,14714,200401,PICARDIE,25,510580,504779,518419,219485,130146,1883409,7,7,7
9192,14715,200401,POITOU-CHARENTES,100,387823,414368,467948,256875,171006,1698020,3,3,3
9193,14716,200401,PROVENCE-ALPES-COTE-D-AZUR,293,1127581,1187177,1279079,691333,433962,4719132,6,6,6
9194,14717,200401,ILE-DE-FRANCE,66,2970969,3493848,3063842,1166374,655257,11350290,7,5,5
9195,14718,200401,RHONE-ALPES,358,1530256,1621106,1586950,738439,427978,5904729,5,4,4


### <span style="color: green;">Description du Dataset</span>

<p>Le jeu de données décrit l'évolution hebdomadaire du nombre de cas de grippe en France par région et en fonction de plusieurs éléments


<p>Il comporte les colonnes suivantes :<br>

• <b>Id</b> : identifiant unique de chaque observation (ligne)<br>
• <b>week</b> : semaine de l'observation au format AAAASS (Année + Numéro de semaine, ex: 200401)<br>
• <b>region_name</b> : nom de la région administrative française (ex: ALSACE, AQUITAINE...)<br>
• <b>TauxGrippe</b> : taux de consultations pour syndromes grippaux pour 100 000 habitants (<b>variable cible</b>)<br>
• <b>pop_0_19</b> à <b>pop_75_plus</b> : estimation de la population par tranche d'âge dans la région pour l'année donnée<br>
• <b>pop_total</b> : population totale de la région pour l'année donnée<br>
• <b>requete_grippe</b> : volume normalisé des recherches Google pour le mot-clé "grippe" dans la région (donnée mensuelle rapportée à la semaine)<br>
• <b>requete_grippe_aviaire_vaccin...</b> : volume des recherches associées aux autres termes (vaccin, grippe aviaire, H1N1...)<br>

In [105]:
# Infos rapides sur le dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9196 entries, 0 to 9195
Data columns (total 13 columns):
 #   Column                                                                              Non-Null Count  Dtype 
---  ------                                                                              --------------  ----- 
 0   Id                                                                                  9196 non-null   int64 
 1   week                                                                                9196 non-null   int64 
 2   region_name                                                                         9196 non-null   object
 3   TauxGrippe                                                                          9196 non-null   int64 
 4   pop_0_19                                                                            9196 non-null   int64 
 5   pop_20_39                                                                           9196 non-null   int6

- Pas de valeurs manquantes. Nos données de panel sont alors cylindrés (équilibrées).

In [106]:
# Conversion de 'week' (int) en véritable datetime
df['week_date'] = pd.to_datetime(df['week'].astype(str) + '1', format='%Y%W%w')# On ajoute '1' pour dire "Lundi de la semaine N"
# Création de la colonne 'year' (int)
df['year'] = df['week_date'].dt.year
# Création de la colonne 'month' (en toutes lettres et en Français)
noms_mois= {
    1: 'Janvier', 2: 'Février', 3: 'Mars', 4: 'Avril',
    5: 'Mai', 6: 'Juin', 7: 'Juillet', 8: 'Août',
    9: 'Septembre', 10: 'Octobre', 11: 'Novembre', 12: 'Décembre'
}
df['month'] = df['week_date'].dt.month.map(noms_mois)
# Vérification
display(df[['week', 'week_date', 'year', 'month']].sample(5))

,week,week_date,year,month
8801,200418,2004-05-03,2004,Mai
56,201150,2011-12-12,2011,Décembre
318,201138,2011-09-19,2011,Septembre
6937,200550,2005-12-12,2005,Décembre
7066,200544,2005-10-31,2005,Octobre


### <span style="color: green;">Identification des types de variables</span>

In [107]:
num_vars, cat_vars = identify_variable_types(df)


Variables numériques (13) :
------------------------------
 - Id
 - week
 - TauxGrippe
 - pop_0_19
 - pop_20_39
 - pop_40_59
 - pop_60_74
 - pop_75_plus
 - pop_total
 - requete_grippe
 - requete_grippe_aviaire_vaccin
 - requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie
 - year

Variables Catégorielles (2) :
------------------------------
 - region_name
 - month


### <span style="color: green;">Visualisations et analyses descriptives</span>

#### <span style="color: green;">Analyse univariée</span>

In [108]:
# Statistiques descriptives des variables numériques
print("Statistiques descriptives - Variables numériques")
print("=" * 60)
df[num_vars].describe()

Statistiques descriptives - Variables numériques


,Id,week,TauxGrippe,pop_0_19,pop_20_39,pop_40_59,pop_60_74,pop_75_plus,pop_total,requete_grippe,requete_grippe_aviaire_vaccin,requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie,year
count,9196.00,9196.00,9196.00,9196.00,9196.00,9196.00,9196.00,9196.00,9196.00,9196.00,9196.00,9196.00,9196.00
mean,10120.50,200776.15,77.55,696445.80,733801.50,768348.14,375331.21,239053.05,2812979.69,9.00,6.09,3.02,2007.50
std,2654.80,229.79,180.94,611297.41,696461.36,623514.35,264212.97,155173.42,2334409.85,16.11,12.89,3.31,2.29
min,5523.00,200401.00,0.00,61737.00,71611.00,79994.00,44827.00,25803.00,283972.00,0.00,0.00,0.00,2004.00
25%,7821.75,200552.00,0.00,360474.00,353576.00,404851.75,215662.00,134465.00,1465926.00,2.00,1.00,1.00,2005.00
50%,10120.50,200776.50,10.00,533070.50,550470.00,589058.00,297194.50,190745.00,2123145.50,3.00,2.00,2.00,2007.50
75%,12419.25,200953.00,59.00,764057.75,773656.00,892061.00,447151.25,299000.00,3200833.75,8.00,5.00,4.00,2010.00
max,14718.00,201152.00,2478.00,3071017.00,3508802.00,3138930.00,1383699.00,773873.00,11852851.00,100.00,86.00,26.00,2011.00


In [109]:
# Configuration de la grille (3 colonnes)
n_cols_subplot = 3
vars = ["pop_0_19", "pop_20_39", "pop_40_59", "pop_60_74", "pop_75_plus", "pop_total", 
        "requete_grippe", "requete_grippe_aviaire_vaccin", 
        "requete_grippe_aviaire_vaccin_porcine_porc_H1N1_AH1N1_A_mexicaine_Mexique_pandemie"]
n_rows_subplot = math.ceil(len(vars) / n_cols_subplot)
# Création des sous-graphiques
# On tronque les titres à 30 caractères pour éviter qu'ils ne se chevauchent
titles = [c[:30] + '...' if len(c) > 30 else c for c in vars]
fig = make_subplots(rows=n_rows_subplot, cols=n_cols_subplot, subplot_titles=titles)
# Boucle pour ajouter chaque histogramme
for i, col in enumerate(vars):
    if col in df.columns:
        row = (i // n_cols_subplot) + 1
        col_idx = (i % n_cols_subplot) + 1
        
        fig.add_trace(
            go.Histogram(x=df[col], name=col),
            row=row, col=col_idx
        )
# Mise en forme finale
fig.update_layout(
    height=300 * n_rows_subplot,  # Hauteur adaptative
    width=1000,
    title_text="Distribution des Variables Numériques",
    showlegend=False,
    bargap=0.1
)
fig.show()

-  Commentaireeeeeeeeeeess

In [110]:
# Visualisation de l'évolution du taux de grippe par région chaque semaine

national_mean = df.groupby('week_date')['TauxGrippe'].mean().reset_index()
fig = go.Figure()

regions = df['region_name'].unique()
for region in regions:
    region_data = df[df['region_name'] == region]
    fig.add_trace(go.Scatter(
        x=region_data['week_date'],
        y=region_data['TauxGrippe'],
        mode='lines',
        name=region,
        opacity=0.3,       # Transparence forte pour ne pas surcharger
        line=dict(width=1),
        showlegend=True    # Garde la légende (cliquable pour filtrer)
    ))

# Ajout de la Moyenne Nationale
fig.add_trace(go.Scatter(
    x=national_mean['week_date'],
    y=national_mean['TauxGrippe'],
    mode='lines',
    name='Moyenne Nationale',
    line=dict(color='black', width=2), # Noir épais
    opacity=0.8
))

# Mise en page
fig.update_layout(
    title='<b>Évolution du Taux de Grippe par Région (2004-2015)</b>',
    xaxis_title='Date',
    yaxis_title='Taux de Grippe (cas / 100k hab.)',
    template='plotly_white',
    hovermode='x unified',  # Très utile : affiche toutes les valeurs au survol de la souris
    height=600,
    legend_title="Régions"
)

fig.show()

In [111]:
# Graphique comparatif : Agrégation Mensuelle vs Annuelle
# Agrégation Mensuelle
df['month_date'] = df['week_date'].dt.to_period('M').dt.to_timestamp()
df_month = df.groupby(['region_name', 'month_date'])['TauxGrippe'].mean().reset_index()
national_mean_month = df_month.groupby('month_date')['TauxGrippe'].mean().reset_index()
# Agrégation Annuelle
df_year = df.groupby(['region_name', 'year'])['TauxGrippe'].mean().reset_index()
national_mean_year = df_year.groupby('year')['TauxGrippe'].mean().reset_index()

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("<b>Évolution Mensuelle Moyenne</b>", "<b>Évolution Annuelle Moyenne</b>"),
    vertical_spacing=0.15
)

regions = df['region_name'].unique()
for region in regions:
    # Données mensuelles
    region_data_m = df_month[df_month['region_name'] == region]
    # Données annuelles
    region_data_y = df_year[df_year['region_name'] == region]
    # Trace Mensuelle (Haut)
    fig.add_trace(go.Scatter(
        x=region_data_m['month_date'],
        y=region_data_m['TauxGrippe'],
        mode='lines',
        name=region,
        opacity=0.3,
        line=dict(width=1),
        legendgroup=region,  # Lie les légendes haut/bas
        showlegend=True      # Affiche la légende une seule fois
    ), row=1, col=1)
    # Trace Annuelle (Bas)
    fig.add_trace(go.Scatter(
        x=region_data_y['year'],
        y=region_data_y['TauxGrippe'],
        mode='lines',
        name=region,
        opacity=0.3,
        line=dict(width=1),
        legendgroup=region,
        showlegend=False     # Masque la légende (déjà affichée par le graphe du haut)
    ), row=2, col=1)
# Moyenne Mensuelle
fig.add_trace(go.Scatter(
    x=national_mean_month['month_date'],
    y=national_mean_month['TauxGrippe'],
    mode='lines',
    name='Moyenne Nationale',
    line=dict(color='black', width=2),
    opacity=0.8,
    legendgroup='Moyenne Nationale'
), row=1, col=1)
# Moyenne Annuelle
fig.add_trace(go.Scatter(
    x=national_mean_year['year'],
    y=national_mean_year['TauxGrippe'],
    mode='lines',
    name='Moyenne Nationale',
    line=dict(color='black', width=2),
    opacity=0.8,
    legendgroup='Moyenne Nationale',
    showlegend=False
), row=2, col=1)

fig.update_layout(
    title='<b>Comparaison : Évolution Mensuelle vs Annuelle du Taux de Grippe</b>',
    template='plotly_white',
    hovermode='x unified',
    height=900,  # Augmenté pour bien voir les deux graphes
    legend_title="Régions"
)
# Ajout des labels d'axes
fig.update_yaxes(title_text="Taux de grippe moyen(cas / 100k)", row=1, col=1)
fig.update_yaxes(title_text="Taux de grippe moyen(cas / 100k)", row=2, col=1)

fig.show()

- Commentaires

In [112]:
# Analyse de la saisonnalité mensuelle du Taux de Grippe (Moyenne sur toutes les années)

monthly_avg = df.groupby('month')['TauxGrippe'].mean()
# Ordre correct pour l'affichage
month_order = ['Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin', 
               'Juillet', 'Août', 'Septembre', 'Octobre', 'Novembre', 'Décembre']
monthly_avg = monthly_avg.reindex(month_order)

# Création du Graphique en Barres
fig = go.Figure(data=[
    go.Bar(
        x=monthly_avg.index,
        y=monthly_avg.values,
        marker_color='black',       # Couleur principale
        marker_line_color='black',  # Contour noir
        marker_line_width=1.5,
        opacity=0.9
    )
])
# Mise en page
fig.update_layout(
    title='<b>Saisonnalité du Taux de Grippe (par mois)</b>',
    xaxis_title='Mois',
    yaxis_title='Taux de Grippe Moyen (cas / 100k)',
    template='plotly_white',
    width=900,
    height=500,
    xaxis=dict(tickangle=-45) # Inclinaison légère des labels si besoin
)
fig.show()
print(" On observe clairement un pic hivernal (décembre-février) caractéristique de l'épidémie de grippe.")

 On observe clairement un pic hivernal (décembre-février) caractéristique de l'épidémie de grippe.


#### <span style="color: green;">Analyse multivariée</span>

In [113]:
# Matrice de Corrélation des Variables Numériques Principales
# Variables pertinentes pour l'analyse
corr_cols = vars + ['TauxGrippe']

corr_matrix = df[corr_cols].corr()
# Abréviation des noms trop longs
# Si le nom fait plus de 30 caractères, on garde les 30 premiers + "..."
short_labels = [col[:30] + '...' if len(col) >= 30 else col for col in corr_matrix.columns]
#short_labels = corr_matrix.columns.tolist()
# Création de la Heatmap Plotly
fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=short_labels,
    y=short_labels,
    colorscale='RdBu',      # Echelle Rouge-Bleu
    reversescale=True,      # On inverse pour avoir Rouge = Corrélation Positive (Hot)
    zmin=-1, zmax=1,        # Bornes fixes entre -1 et 1
    text=corr_matrix.values, # Les valeurs brutes
    texttemplate="%{text:.2f}", # Format d'affichage (2 décimales) sur les cases
    textfont={"size": 10}
))
# Mise en forme
fig.update_layout(
    title='<b>Matrice de Corrélation des Variables Principales</b>',
    width=900,
    height=800,
    template='plotly_white',
    xaxis=dict(tickangle=-45) # Inclinaison des labels pour la lisibilité
)

fig.show()

* **`TauxGrippe` vs `requete_grippe` (Google Trends)** : On note une corrélation forte de 0.39.
Il y a un lien fort entre ce que les gens tapent sur Google ("grippe") et le nombre de malades réels. Mais attention, a priori c'est de la corrélation et pas nécéssairement de la causalité.

* **`TauxGrippe` vs Populations** : La corrélation est quasi-nulle. 
Le taux de grippe (malades pour 100k habitants) ne dépend pas a priori de la taille de la région. Une petite région comme la Corse peut avoir un taux d'infection aussi élevé que l'Île-de-France. La taille de la population n'aide pas directement à prédire l'intensité de l'épidémie (le taux), mais le volume de malades.

* Le "Bloc Rouge" des **Populations** présente des corrélations très proches de 1 et c'est logique. Une région très peuplée a *beaucoup* de jeunes, *beaucoup* d'adultes et *beaucoup* de seniors. Ces variables racontent toutes la même histoire : "La taille de la région". On pourrait ne garder que `pop_total` et peut-être une colonne de ratio (ex: pourcentage de seniors) et supprimer les autres pour éviter de la **multicollinéarité**.

* Les autres **requêtes Google** (`grippe_aviaire`, `H1N1`...) très correlées entre elles car elles sont toutes liées au mot "grippe". La variable `requete_grippe` contient les occurences de la variable `requete_grippe_aviaire` qui contient celles de la dernière.

### <span style="color: green;">Encodage des variables</span>

### <span style="color: green;">Sauvegarde du dataset</span>

In [114]:
# Sauvegarde du dataset nettoyé (optionnel)
df.to_csv('../data/processed/train_final.csv', index=False)

### <span style="color: green;">Fin</span>